# SegNet implementation

In [50]:
import sys
import os
import matplotlib.pyplot
import numpy as np
import tensorflow as tf
import cv2
import random

Convolution layer. takes parameters; input tensor, number of filters and kernel size

In [23]:
def conv_layer(input, filters, kernel_size = 7):
    return tf.layers.conv2d(
            inputs = input,
            filters = filters,
            kernel_size = [kernel_size, kernel_size],
            padding = "same",
            activation = tf.nn.relu
            )

Convolution layer. takes parameters; input tensor, number of filters. Same as the Convolution layer except that it neglects the biases and does not apply activation function

In [24]:
def deconv_layer(input,filters):
    return tf.layers.conv2d(
            inputs = input,
            filters = filters,
            kernel_size = [7,7],
            padding = "same",
            use_bias=False
            )

Max pool layer which takes input tensor and returns a tensor with half size and map of the max pooled tensor

In [25]:
def maxpool_layer(input):
    return(tf.nn.max_pool_with_argmax(input, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME'))

The unpool layer which takes an input tensor and the map of indices and returns the unpooled tensor of double size of the input

In [26]:
def unpool_layer(input, index, ksize=[1, 2, 2, 1]):
    input_shape = input.get_shape().as_list()
    output_shape = (input_shape[0], input_shape[1] * ksize[1], input_shape[2] * ksize[2], input_shape[3])

    flat_output_shape = [output_shape[0], output_shape[1] * output_shape[2] * output_shape[3]]

    uppool = tf.reshape(input, [np.prod(input_shape)])
    batch_range = tf.reshape(tf.range(output_shape[0], dtype=index.dtype), shape=[input_shape[0], 1, 1, 1])
    b = tf.ones_like(index) * batch_range
    b = tf.reshape(b, [np.prod(input_shape), 1])
    ind = tf.reshape(index, [np.prod(input_shape), 1])
    ind = tf.concat([b, ind], 1)

    output = tf.scatter_nd(ind, uppool, shape=flat_output_shape)
    return tf.reshape(output, output_shape)


The main model of Segnet which consists of 13 Convolution layer, 5 max pooling layers, 13 deconvloution layers and 5 unpooling layers. Attached to pixelwise classifier. 
We assumed that we only have two classes.
Still missing the loss function

In [65]:
def model(input, scope="SegNet", reuse=True):
    input = tf.reshape(input["x"], [-1, 256, 256, 1])
    n_labels = 2
    with tf.name_scope(scope):
        # first layer
        conv_1 = conv_layer(input, 64)
        conv_2 = conv_layer(conv_1, 64)
        pool_1, indicies_1 = maxpool_layer(conv_2)
        
        # second layer
        conv_3 = conv_layer(pool_1, 128)
        conv_4 = conv_layer(conv_3, 128)
        pool_2, indicies_2 = maxpool_layer(conv_4)
        
        #third layer
        conv_5 = conv_layer(pool_2, 256)
        conv_6 = conv_layer(conv_5, 256)
        conv_7 = conv_layer(conv_6, 256)
        pool_3, indicies_3 = maxpool_layer(conv_7)
        
        #fourth layer
        conv_8 = conv_layer(pool_3, 512)
        conv_9 = conv_layer(conv_8, 512)
        conv_10 = conv_layer(conv_9, 512)
        pool_4, indicies_4 = maxpool_layer(conv_10)
        
        #fifth layer
        conv_11 = conv_layer(pool_4, 512)
        conv_12 = conv_layer(conv_11, 512)
        conv_13 = conv_layer(conv_12, 512)
        pool_5, indicies_5 = maxpool_layer(conv_13)
        
        
        # the decoding layers
        # fifth layer
        unpool_5 = unpool_layer(pool_5, indicies_5)
        deconv_13 = deconv_layer(unpool_5, 512)
        deconv_12 = deconv_layer(deconv_13, 512)
        deconv_11 = deconv_layer(deconv_12, 512)
       
        
        # forth layer
        unpool_4 = unpool_layer(deconv_11, indicies_4)
        deconv_10 = deconv_layer(unpool_4, 512)
        deconv_9 = deconv_layer(deconv_10, 512)
        deconv_8 = deconv_layer(deconv_9, 256)
        
        
        # third layer
        unpool_3 = unpool_layer(deconv_8, indicies_3)
        deconv_7 = deconv_layer(unpool_3, 256)
        deconv_6 = deconv_layer(deconv_7, 256)
        deconv_5 = deconv_layer(deconv_6, 128)
        
        # second layer
        unpool_2 = unpool_layer(deconv_5, indicies_2)
        deconv_4 = deconv_layer(unpool_2, 128)
        deconv_3 = deconv_layer(deconv_4, 64)
        
        # first layer
        unpool_1 = unpool_layer(deconv_3, indicies_1)
        deconv_2 = deconv_layer(unpool_1, 64)
        #deconv_1 = deconv_layer(deconv_2, 64)
        
        # Classification
        classifier_layer = conv_layer(deconv_2, n_labels, kernel_size = 1)
        softmax = tf.nn.softmax(classifier_layer)
        
        onehot_labels = tf.one_hot(indices= inputs["y"], depth = 2)
        loss = tf.losses.softmax_cross_entropy(onehot_labels = onehot_labels, logits = classifier_layer)
        print(loss)
        print(softmax.shape)


Test that the model is working properly with the liver input image

In [74]:
im = cv2.imread('liver.jpeg', cv2.IMREAD_GRAYSCALE)
im = cv2.resize(im,(256,256))
im = np.array(im, dtype=np.float32)
im = np.expand_dims(im, axis = 0)
im = np.expand_dims(im, axis = 3)
labels = [[random.randint(0,1) for i in range(256)] for j in range(256)]
labels = np.array(labels)
prizzz
inputs = {"x": im, "y": labels}
model(inputs)



(256, 256)


ValueError: Shapes (1, 256, 256, 2) and (256, 256, 2) are incompatible